In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy

In [2]:
df = pd.read_csv("./data/train.csv")

In [3]:
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [6]:
nlp = spacy.load('en_core_web_sm')

vecs1 = []
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        vec1 = word1.vector        
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis = 0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)

100%|████████████████████████████████| 404290/404290 [7:57:01<00:00, 14.13it/s]


In [8]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        vec2 = word2.vector
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|████████████████████████████████| 404290/404290 [8:33:48<00:00, 13.11it/s]


In [10]:
df_nlp = pd.read_csv("./data/train_features_nlp.csv", encoding = 'latin-1')
df_basic = pd.read_csv("./data/train_basic_features.csv",encoding = 'latin-1')

In [11]:
df1 = df_nlp.drop(['qid1','qid2','question1','question2'], axis = 1)
df2 = df_basic.drop(['qid1','qid2','question1','question2','is_duplicate'], axis = 1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'], axis = 1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index = df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index = df3.index)

In [12]:
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [13]:
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [14]:
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,211.129893,-144.683061,-68.811244,-153.662140,-89.931597,2.311302,136.743758,50.449081,-64.150966,56.627547,...,33.176578,-102.486987,144.595711,50.614239,-21.385917,36.385631,-70.999799,4.141477,-131.379516,71.884593
1,144.124634,-114.012491,-111.716680,-104.885037,-88.238481,16.441812,58.238031,102.095111,6.027000,178.498484,...,67.386277,22.131792,40.210776,0.254025,98.600192,-36.436016,-123.609740,71.788340,-15.374613,-10.503979
2,81.757899,-142.184499,0.559856,-104.660094,-84.156634,22.515108,115.521638,50.436934,-111.740905,51.713317,...,61.731852,-49.822520,107.080607,-11.431434,-40.667738,-26.771792,-12.482070,1.764708,-100.330478,148.061208
3,-126.651911,-59.747171,-67.763215,-138.114741,-101.038710,88.148499,-22.912265,85.941412,27.784230,50.810660,...,117.847465,-13.044516,-1.643163,78.158409,23.748114,12.945965,-8.807305,123.360097,64.057023,-24.943296
4,299.444081,-188.631976,-22.946297,-273.683323,-188.480368,107.123113,174.946323,-72.042336,-98.290461,137.439943,...,62.256707,-173.503681,249.320194,27.972506,-26.665114,-28.469195,-62.335430,2.738913,-120.762673,111.779861


In [15]:
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,151.268542,-127.013175,-31.546276,-142.905818,-97.249098,9.485766,106.682268,36.754171,-36.541898,53.162222,...,28.763386,-108.306615,131.115603,42.530926,-14.726218,15.763673,-59.729539,-9.563379,-99.812542,57.643648
1,152.023059,-44.955377,-103.559249,-128.467573,-118.567593,44.577941,137.906161,26.984767,-78.328365,86.576861,...,117.588047,5.239809,56.622513,92.054120,141.063318,-30.466189,-25.418969,87.120100,-23.594520,7.936903
2,4.930235,-29.029568,-117.808800,-98.332304,-19.064150,-9.867795,141.808210,91.269551,50.727192,12.816859,...,128.699940,17.433408,58.203366,88.642932,-84.962777,71.462348,-10.853923,-59.501677,-99.425349,95.334779
3,-6.951911,-44.951746,-17.343061,-61.444448,-7.469157,16.941989,95.049244,-2.631607,-13.050923,-28.038392,...,70.475245,-31.174270,86.080315,49.967737,25.750446,-27.612384,24.050796,-3.504149,-46.115803,56.242293
4,96.174553,-71.613974,21.584887,-92.742456,-106.643117,10.646815,92.190149,-40.565996,-34.739526,56.340522,...,18.852784,-70.416571,87.693603,38.386015,-14.003132,-12.975111,46.318839,-25.159628,-53.098675,-2.351913


In [16]:
df3_q1['id'] = df1['id']
df3_q2['id'] = df1['id']
df1  = df1.merge(df2, on = 'id', how = 'left')
df2  = df3_q1.merge(df3_q2, on = 'id',how = 'left')
result  = df1.merge(df2, on = 'id', how = 'left')
result.to_csv('./data/train_final_features.csv')